In [1]:
# %run ./footprint_v_biocapacity.ipynb
import pandas as pd

filepath = "Resources/countries.csv"
eco_csv = pd.read_csv(filepath, header=0)
eco_csv.head()
# eco_csv.set_index("Region")

my_df = pd.DataFrame(eco_csv.iloc[:,[0,1,5,6,7,8,9,10,11,12,13,14,15,16,17]])
my_df.head()

,Country,Region,Cropland Footprint,Grazing Footprint,Forest Footprint,Carbon Footprint,Fish Footprint,Total Ecological Footprint,Cropland,Grazing Land,Forest Land,Fishing Water,Urban Land,Total Biocapacity,Biocapacity Deficit or Reserve
0,Afghanistan,Middle East/Central Asia,0.30,0.20,0.08,0.18,0.00,0.79,0.24,0.20,0.02,0.00,0.04,0.50,-0.30
1,Albania,Northern/Eastern Europe,0.78,0.22,0.25,0.87,0.02,2.21,0.55,0.21,0.29,0.07,0.06,1.18,-1.03
2,Algeria,Africa,0.60,0.16,0.17,1.14,0.01,2.12,0.24,0.27,0.03,0.01,0.03,0.59,-1.53
3,Angola,Africa,0.33,0.15,0.12,0.20,0.09,0.93,0.20,1.42,0.64,0.26,0.04,2.55,1.61
4,Antigua and Barbuda,Latin America,NaN,NaN,NaN,NaN,NaN,5.38,NaN,NaN,NaN,NaN,NaN,0.94,-4.44


# Countries with Highest Biodeficit

In [2]:
my_df['Biocapacity Deficit or Reserve %'] = my_df['Biocapacity Deficit or Reserve']/my_df['Total Biocapacity'] * 100
eco_sort = my_df.sort_values('Biocapacity Deficit or Reserve %')
eco_sort.reset_index(inplace=True, drop=True)
eco_sort.head()


,Country,Region,Cropland Footprint,Grazing Footprint,Forest Footprint,Carbon Footprint,Fish Footprint,Total Ecological Footprint,Cropland,Grazing Land,Forest Land,Fishing Water,Urban Land,Total Biocapacity,Biocapacity Deficit or Reserve,Biocapacity Deficit or Reserve %
0,Singapore,Asia-Pacific,0.67,0.24,0.91,5.91,0.22,7.97,0.00,0.00,0.00,0.01,0.03,0.05,-7.92,-15840.000000
1,Bermuda,North America,NaN,NaN,NaN,NaN,NaN,5.77,NaN,NaN,NaN,NaN,NaN,0.13,-5.64,-4338.461538
2,Barbados,Latin America,0.56,0.24,0.14,3.28,0.22,4.48,0.08,0.00,0.02,0.04,0.04,0.19,-4.29,-2257.894737
3,Aruba,Latin America,NaN,NaN,NaN,NaN,NaN,11.88,NaN,NaN,NaN,NaN,NaN,0.57,-11.31,-1984.210526
4,Réunion,Africa,0.14,0.01,0.15,3.27,0.09,3.65,0.14,0.01,0.04,0.00,0.00,0.18,-3.47,-1927.777778


# Countries with Highest Bioreserve

In [3]:
eco_sort.tail()

,Country,Region,Cropland Footprint,Grazing Footprint,Forest Footprint,Carbon Footprint,Fish Footprint,Total Ecological Footprint,Cropland,Grazing Land,Forest Land,Fishing Water,Urban Land,Total Biocapacity,Biocapacity Deficit or Reserve,Biocapacity Deficit or Reserve %
183,Congo,Africa,0.22,0.09,0.38,0.47,0.10,1.29,0.13,3.06,7.33,0.35,0.03,10.91,9.63,88.267644
184,Gabon,Africa,0.53,0.13,0.79,0.43,0.11,2.02,0.25,3.60,19.50,2.93,0.03,26.31,24.29,92.322311
185,Suriname,Latin America,0.43,0.06,0.52,3.01,0.14,4.25,0.30,0.03,81.52,7.39,0.09,89.33,85.08,95.242360
186,Guyana,Latin America,0.70,0.22,0.77,1.25,0.06,3.07,0.69,1.51,59.19,5.12,0.06,66.58,63.51,95.389006
187,French Guiana,Latin America,0.07,0.06,0.46,1.58,0.17,2.34,0.07,0.06,95.16,16.07,0.00,111.35,109.01,97.898518


# How U.S. ranks

In [4]:
index_us = eco_sort.loc[eco_sort["Country"] == "United States of America",:]
index_us


,Country,Region,Cropland Footprint,Grazing Footprint,Forest Footprint,Carbon Footprint,Fish Footprint,Total Ecological Footprint,Cropland,Grazing Land,Forest Land,Fishing Water,Urban Land,Total Biocapacity,Biocapacity Deficit or Reserve,Biocapacity Deficit or Reserve %
71,United States of America,North America,1.13,0.3,0.67,5.9,0.12,8.22,1.49,0.28,1.57,0.33,0.09,3.76,-4.46,-118.617021


# Regional Totals

In [9]:
region = eco_sort.groupby('Region').sum()
region['Biocapacity Deficit or Reserve %'] = region['Biocapacity Deficit or Reserve']/region['Total Biocapacity'] * 100
region.sort_values('Biocapacity Deficit or Reserve %', ascending=False)

,Cropland Footprint,Grazing Footprint,Forest Footprint,Carbon Footprint,Fish Footprint,Total Ecological Footprint,Cropland,Grazing Land,Forest Land,Fishing Water,Urban Land,Total Biocapacity,Biocapacity Deficit or Reserve,Biocapacity Deficit or Reserve %
Region,,,,,,,,,,,,,,
Latin America,14.85,12.57,11.91,56.41,4.37,135.87,15.26,19.18,282.18,46.42,1.80,372.98,237.14,63.579817
Africa,19.48,11.48,15.50,29.04,4.69,86.89,15.20,29.78,51.11,18.19,1.98,117.66,30.78,26.160122
Asia-Pacific,17.96,7.97,11.22,45.42,6.97,98.02,16.51,18.07,35.48,18.09,2.29,93.29,-4.72,-5.059492
North America,2.59,0.63,1.87,10.90,0.24,22.16,4.60,0.57,10.69,3.74,0.16,19.90,-2.27,-11.407035
Northern/Eastern Europe,7.79,1.76,5.07,24.32,0.66,45.16,7.56,1.95,14.34,2.04,0.55,34.59,-10.57,-30.557965
European Union,23.59,6.25,15.40,78.34,2.78,142.36,25.96,2.92,29.36,9.39,3.25,94.87,-47.50,-50.068515
Middle East/Central Asia,13.77,4.87,3.70,67.82,1.48,93.25,6.92,6.52,2.28,5.09,1.58,22.41,-70.87,-316.242749


# Regional Land Type Usage

In [10]:
#     region['Cropland Footprint'] / region['Cropland'] * 100

region_landuse = pd.DataFrame({
    'Cropland Deficit or Reserve' : region['Cropland'] - region['Cropland Footprint'],
#     'Cropland Deficit or Reserve %' : region['Cropland'] - region['Cropland Footprint'],
    'Grazing Deficit or Reserve' : region['Grazing Land'] - region['Grazing Footprint'],
    'Forest Deficit or Reserve' : region['Forest Land'] - region['Forest Footprint'],
    'Carbon Deficit or Reserve' : region['Fishing Water'] - region['Carbon Footprint'],
    'Fish Deficit or Reserve' : region['Urban Land'] - region['Fish Footprint'],
    'Total Bio Deficit or Reserve' : region['Biocapacity Deficit or Reserve'],
})

def highlight_pos(c):
    is_pos = c >= 0
    return ['background-color: green' if v else 'red' for v in is_pos]
region_landuse.style.apply(highlight_pos)


,Cropland Deficit or Reserve,Grazing Deficit or Reserve,Forest Deficit or Reserve,Carbon Deficit or Reserve,Fish Deficit or Reserve,Total Bio Deficit or Reserve
Region,,,,,,
Africa,-4.28,18.3,35.61,-10.85,-2.71,30.78
Asia-Pacific,-1.45,10.1,24.26,-27.33,-4.68,-4.72
European Union,2.37,-3.33,13.96,-68.95,0.47,-47.5
Latin America,0.41,6.61,270.27,-9.99,-2.57,237.14
Middle East/Central Asia,-6.85,1.65,-1.42,-62.73,0.1,-70.87
North America,2.01,-0.06,8.82,-7.16,-0.08,-2.27
Northern/Eastern Europe,-0.23,0.19,9.27,-22.28,-0.11,-10.57
